In [29]:
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', 100)

In [1309]:
flat = pd.read_csv('input/flat.csv', encoding='cp1251', index_col=['bulk_id', 'spalen', 'id_flatwork'], 
                   parse_dates=['date_settle', 'date_salestart', 'flat_startsale', 'sale']).sort_index()

In [1156]:
train = pd.read_csv('input/train.csv', encoding='cp1251', index_col=['bulk_id', 'spalen', 'date1'], parse_dates=['date1']).sort_index()

In [1158]:
test = pd.read_csv('input/test.csv', encoding='cp1251', index_col=['bulk_id', 'spalen', 'date1'], parse_dates=['date1']).sort_index()

In [1284]:
test['value'] = 0

In [1285]:
train_test = pd.concat([train, test])

In [1159]:
status = pd.read_csv('input/status.csv',  encoding='cp1251', index_col=['id_flatwork'], parse_dates=['datefrom', 'dateto']).sort_index()

In [1160]:
price = pd.read_csv('input/price.csv', index_col=['id_flatwork'], parse_dates=['datefrom']).sort_values('datefrom', ascending=False)

In [1165]:
min_date = pd.to_datetime('2015-07-01')
max_date = pd.to_datetime('2018-05-01')

dates = pd.date_range(min_date, max_date, freq=pd.offsets.MonthBegin(normalize=True))

In [1166]:
prices = dates.map(lambda date: pd.DataFrame({'date1': date, 
                                              'pricem2': price[price['datefrom'] < date].groupby(level=0)['pricem2'].first()}))

In [1167]:
prices = pd.concat(prices)

In [1168]:
stat_active = [100000001, 100000003, 100000006, 100000010, 100000020, 100000021]
stat_reserved = [100000003, 100000020, 100000021] # ['Зарезервирован под клиента', 'Платное бронирование', 'Онлайн бронирование']

In [1169]:
status_active = status[status['stat'].isin(stat_active)]
status_reserved = status[status['stat'].isin(stat_reserved)]

In [1170]:
statuses_active = pd.concat(dates.map(lambda date: pd.DataFrame(
    {'date1': date, 
     'stat': status_active[status_active['datefrom'] < date].groupby(level=0)['stat'].first()})))
statuses_reserved = pd.concat(dates.map(lambda date: pd.DataFrame(
    {'date1': date, 
     'stat': status_reserved[status_reserved['datefrom'] < date].groupby(level=0)['stat'].first()})))

In [1310]:
flat_prices = pd.merge(flat.reset_index()[['bulk_id', 'spalen', 'id_flatwork', 'floor', 'square', 'flat_startsale', 'sale']], 
                       prices.reset_index(), on=['id_flatwork'], how='left')

flat_prices = flat_prices[flat_prices['pricem2'] != 1.0]

flat_prices = flat_prices[(flat_prices['flat_startsale'] <= flat_prices['date1']) & 
            (flat_prices['sale'] > flat_prices['date1'])]

flat_active = pd.merge(flat_prices, 
                     statuses_active.reset_index(), on=['id_flatwork', 'date1'], how='left').dropna()
flat_reserved = pd.merge(flat_prices, 
                     statuses_reserved.reset_index(), on=['id_flatwork', 'date1'], how='left').dropna()

In [1311]:
flat_active.to_csv('output/flat_active.csv', index=False)
flat_reserved.to_csv('output/flat_reserved.csv', index=False)

In [1312]:
mean_prices = flat_prices.groupby('id_flatwork')['pricem2'].mean()

In [1313]:
def aggregate(df):
    names = {'flats_left': len(df), 
             'max_fl': df['floor'].max(),
             'min_fl': df['floor'].min(),
             'mean_fl': df['floor'].mean(),
             'max_sq': df['square'].max(),
             'min_sq': df['square'].min(),
             'sum_sq': df['square'].sum(),
             'mean_sq': df['square'].mean(),
             'max_price': df['pricem2'].max(),
             'min_price': df['pricem2'].min(),
             'mean_price': df['pricem2'].mean()}
    return pd.Series(names)

In [1314]:
flat['months_to_sale'] = flat.groupby(level=[0, 1]) \
.apply(lambda df: (df['sale'].sort_values().dt.to_period('M') - df['sale'].min().to_period('M'))) \
.reset_index([0, 1], drop=True)

In [1315]:
flat = flat.join(mean_prices).dropna(subset=['pricem2', 'months_to_sale'])

In [1316]:
flat[flat['sale'] != flat['sale'].max()].reset_index() \
  .to_csv('input/flat_train.csv', index=False, columns=pd.read_csv('input/flat.csv', encoding='cp1251').columns.drop('sale').tolist() + ['pricem2', 'months_to_sale'])

flat[flat['sale'] == flat['sale'].max()].reset_index() \
  .to_csv('input/flat_test.csv', index=False, columns=pd.read_csv('input/flat.csv', encoding='cp1251').columns.drop('sale').tolist() + ['pricem2', 'months_to_sale'])

In [1317]:
def calc_flat_features(df):
    flat_features = df.groupby(['bulk_id', 'spalen', 'date1']).apply(aggregate).sort_index()

    flat_features['flats_sold'] = flat_features.groupby(level=[0,1])['flats_left'].apply(lambda x: x[::-1].diff()[::-1].abs())
    flat_features['flats_value'] = flat_features.groupby(level=[0,1])['sum_sq'].apply(lambda x: x[::-1].diff()[::-1].abs())

    flat_features['flats_sold_lag'] = flat_features.groupby(level=[0,1])['flats_sold'].apply(lambda x: x.shift(1))
    flat_features['flats_value_lag'] = flat_features.groupby(level=[0,1])['flats_value'].apply(lambda x: x.shift(1))

    flat_features.index.set_levels(flat_features.index.levels[1].astype(int), level=1, inplace=True)

    flat_features = flat_features.reindex(train_test.index)

    flat_features.loc[train_test.index, 'value'] = train_test['value']

    flat_features['value_lag'] = flat_features.groupby(level=[0,1])['value'].apply(lambda x: x.shift(1)).fillna(0)

    flat_features.loc[:, ['flats_sold_lag', 'flats_value_lag']] = flat_features.loc[:, ['flats_sold_lag', 'flats_value_lag']].fillna(0)
    
    return flat_features

In [1318]:
flat_features_active = calc_flat_features(flat_active)
flat_features_reserved = calc_flat_features(flat_reserved)

/Users/nmurzin/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/zmqshell.py:537: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)


In [1319]:
flat_features = pd.merge(flat_features_active.reset_index(), flat_features_reserved.reset_index(), 
                         on=['bulk_id', 'spalen', 'date1'], suffixes=('_active', '_reserved'))

In [1320]:
flat_features.to_csv('output/flat_features.csv')

In [1321]:
train_test_2 = pd.merge(train_test.reset_index(), flat_features.reset_index(), on=['bulk_id', 'spalen', 'date1'], how='left')

In [1322]:
columns = pd.read_csv('input/train.csv', encoding='cp1251').drop(
    ['start_square', 'price', 'mean_fl', 'mean_sq', 'plan_s', 'plan_m', 'plan_l', 'vid_0', 'vid_1', 'vid_2'], axis=1).columns.tolist() \
    + [c + suffix for c in ['flats_left', 'max_fl', 'max_sq', 'min_fl', 'min_sq', 'mean_fl', 'mean_sq', 'sum_sq', 'max_price', 'min_price', 'mean_price',
       'flats_sold', 'flats_sold_lag', 'value_lag'] for suffix in ['_active', '_reserved']]

In [1323]:
train_test_2['date1'] = train_test_2.date1.apply(lambda x: int(x.timestamp()))

In [1324]:
train_test_2[train_test_2['date1'] <= pd.to_datetime('2018-02-01').timestamp()].sort_values('id').fillna(0).to_csv('input/train2.csv', index=False, columns=columns)
# train2[train2['date1'] < pd.to_datetime('2017-12-01').timestamp()].sort_values('id').fillna(0).to_csv('input/train2_train.csv', index=False, columns=columns)
# train2[train2['date1'] >= pd.to_datetime('2017-12-01').timestamp()].sort_values('id').fillna(0).to_csv('input/train2_val.csv', index=False, columns=columns)

train_test_2.loc[train_test_2['value'].isnull(), 'value'] = 0
train_test_2[train_test_2['date1'] > pd.to_datetime('2018-02-01').timestamp()].sort_values('id').fillna(0).to_csv('input/test2.csv', index=False, columns=columns)